In [14]:
import os
from matplotlib import pyplot as plt
import numpy as np
import h5py
import pandas as pd
from pyproj import Proj

In [48]:
df = pd.read_pickle('data/all-power-1.pkl')

In [4]:
def indicesForCoord(coords, lat_index, lon_index):
    dset_coords = coords
    projstring = """+proj=lcc +lat_1=30 +lat_2=60 
                    +lat_0=38.47240422490422 +lon_0=-96.0 
                    +x_0=0 +y_0=0 +ellps=sphere 
                    +units=m +no_defs """
    projectLcc = Proj(projstring)
    origin_ll = reversed(dset_coords[0][0])  # Grab origin directly from database
    origin = projectLcc(*origin_ll)
    
    coords = (lon_index,lat_index)
    coords = projectLcc(*coords)
    delta = np.subtract(coords, origin)
    ij = [int(round(x/2000)) for x in delta]
    return tuple(reversed(ij))

In [5]:
coords = np.load('nrel-coords.npy')

In [49]:
nrel_indexs = [indicesForCoord(coords, lat, lon) for (lat,lon) in zip(df['latitude'],df['longitude'])]

In [52]:
np.count_nonzero([x[1] <= 0 for x in nrel_indexs])

0

In [53]:
nrel_i0 = [x[0] for x in nrel_indexs]
nrel_i1 = [x[1] for x in nrel_indexs]

In [103]:
len(nrel_indexs)

993

In [113]:
wind_list = []
for idx,(i,j) in enumerate(nrel_indexs[:]):
    print(idx)
    wind_list.append(nrel['f']['windspeed_80m'][:,i,j])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248


OSError: Error retrieving data: 429

In [115]:
#fnp.save('data/all-wind-1/wind/all_time0:248.npy',np.array(wind_list).astype(np.float))

In [110]:
61368*len(nrel_indexs)*(64/8/1000/1000)

487.507392

In [71]:
import imp
imp.reload(wind)
import wind
#nrel = wind.setup_nrel()

In [75]:
features = ['elevation','mean_wind_speed', 'pow_curve','temperature']

In [106]:
feat_list = []
for idx,(i,j) in enumerate(nrel_indexs[900:993]):
    print(idx)
    feat_list.append(wind.idx_to_features(nrel,coords,(i,j),features,year = 2013))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


In [105]:
len(feat_list)

200

In [107]:
#fnp.save('data/all-wind-1/features900:993.npy',np.array(feat_list).astype(np.float))

In [104]:
#np.count_nonzero(np.isnan(np.load('data/all-wind-1/features700:900.npy')))

2

In [35]:
np_data = f['windspeed_80m'][:,nrel_i0[1:3],nrel_i1[1:3]]

TypeError: Indexing elements must be in increasing order

In [28]:
f['windspeed_80m'].shape

(792, 1602, 2976)

In [108]:
nrel['f']['windspeed_80m'].shape

(61368, 1602, 2976)

487.507392

In [29]:
nrel_indexs[1]

(788, 1209)

In [27]:
nrel_indexs[1]

(788, 1209)

In [26]:
np_data.shape

(792, 2, 2976)

In [22]:
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
for mo in '01,03,04,05,06,07,08,09,10,11,12'.split(','):
    name = '2013-' + mo
    f = h5py.File('/Users/kook/mount/wtk-us/' + name + '.h5', 'r')
    #f = h5py.File('/Users/kook/mount/conus/v1.1.0/wtk_conus_2014.h5', 'r')
    #f = h5py.File('/Users/kook/mount/conus/v1.1.0/2014/wtk_conus_2014_80m.h5', 'r')
    c
    np.save('data/all-wind-1/wind/' + name + '.npy',np_data)

TypeError: Indexing elements must be in increasing order